>Get the model :

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATConv.1.model.single_batch.2607.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATv2.1.model.single_batch.2707.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023


In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, HeteroConv , GATConv, GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore")


In [2]:
path_work = "/media/concha-eloko/Linux/PPT_clean"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
    # Open the embeddings
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )
DF_embeddings.rename(columns={0: 'index'}, inplace=True)

    # Filter the DF :
DF_info_filtered = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_ToReLabel = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]
all_data = pd.merge(DF_info_filtered , DF_embeddings , on = "index")

# Mind the over representation of outbreaks :
all_data = all_data.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)
all_data

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.030484,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339
1,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__150,anubis,anubis__0,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,prophage_12186,0.036016,0.005938,...,0.008465,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658
2,GCF_016651625.1__phage12,KL43,GCF_016651625.1,GCF_016651625.1__phage12__59,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_924,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
3,GCF_019928025.1__phage0,KL43,n1471,GCF_019928025.1__phage0__10,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_2929,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
4,GCF_004313505.1__phage4,KL14,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,-0.003019,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672,GCF_015209405.1__phage17,KL2,81.9/1004099,GCF_015209405.1__phage17__0,anubis,anubis__1643,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,prophage_12,-0.014377,0.119705,...,0.040801,-0.001167,0.108840,0.008335,0.026747,0.046616,0.033648,-0.092224,0.048720,0.034757
9673,GCF_004311345.1__phage11,KL34,GCF_004311345.1,GCF_004311345.1__phage11__99,anubis,anubis__1644,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11944,-0.001691,-0.067717,...,0.119783,0.073706,0.057788,0.004341,0.007389,-0.081588,0.100995,-0.045545,0.021685,0.013167
9674,GCF_900172635.1__phage3,KL124,GCF_900172635.1,GCF_900172635.1__phage3__1811,anubis,anubis__1647,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11522,-0.002816,-0.055336,...,0.110956,0.056314,0.059700,0.004641,0.016676,-0.071711,0.095491,-0.055784,0.015615,0.000201
9675,GCF_013604525.1__phage18,KL30,n49324932,GCF_013604525.1__phage18__52,anubis,anubis__1649,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...,prophage_2576,-0.000031,0.046085,...,-0.016720,-0.006530,0.077754,-0.024675,0.026408,0.016113,0.043904,-0.091418,0.078778,0.105842


In [3]:
indexation_nodes_A = all_data["Infected_ancestor"].unique().tolist()  
indexation_nodes_B1 = all_data["Phage"].unique().tolist() + [f"Dpo_to_predict_{n}" for n in DF_embeddings["index"].unique().tolist()]
indexation_nodes_B2 = DF_embeddings["index"].unique().tolist() 

ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}

ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}

ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}



In [5]:
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
index_interest = []
for ancestor in instances_bacteria.Infected_ancestor :
    index = ID_nodes_A[ancestor]
    index_interest.append(index)


In [6]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.2607.OHE.pt')
graph_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[11339, 0] },
  B2={ x=[3608, 1280] },
  (B1, infects, A)={
    edge_index=[2, 7731],
    y=[7731]
  },
  (B2, expressed, B1)={
    edge_index=[2, 13285],
    y=[13285]
  },
  (A, harbors, B1)={
    edge_index=[2, 7731],
    y=[7731]
  }
)

In [7]:
class GNN(torch.nn.Module):
    def __init__(self, edge_type , conv, hidden_channels, heads, dropout): 
        super().__init__()
        self.conv = conv((-1,-1), hidden_channels, add_self_loops = False, heads = heads, dropout = dropout, shared_weights = True)
        self.hetero_conv = HeteroConv({edge_type: self.conv})
    def forward(self, x_dict, edge_index_dict):
        x = self.hetero_conv(x_dict, edge_index_dict)
        return x

# Classifier, Binary :
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        self.lin1 = torch.nn.Linear(heads*hidden_channels + 127, 512)
        self.lin2 = torch.nn.Linear(512, 1)

    def forward(self, x_dict_A , x_dict_B1, graph_data):
        edge_type = ("B1", "infects", "A")
        edge_feat_A = x_dict_A["A"][graph_data[edge_type].edge_label_index[1]]
        edge_feat_B1 = x_dict_B1["B1"][graph_data[edge_type].edge_label_index[0]]
        features_phage = torch.cat((edge_feat_A ,edge_feat_B1), dim=-1)
        x = self.lin1(features_phage).relu()
        x = self.lin2(x)
        return x.view(-1)

class Model(torch.nn.Module):
    def __init__(self, conv, hidden_channels, heads, dropout):
        super().__init__()
        self.single_layer_model = GNN(("B2", "expressed", "B1") ,conv, hidden_channels,heads,dropout)
        self.EdgeDecoder = EdgeDecoder(hidden_channels,heads)

    def forward(self, graph_data):
        b1_nodes = self.single_layer_model(graph_data.x_dict , graph_data.edge_index_dict)
        a_nodes =  graph_data.x_dict
        out = self.EdgeDecoder(a_nodes ,b1_nodes , graph_data)
        return out


In [8]:
@torch.no_grad()
def make_predictions(model, data):
    model.eval() 
    output = model(data)
    probabilities = torch.sigmoid(output)  # Convert output to probabilities
    predictions = probabilities.round()  # Convert probabilities to class labels
    return predictions, probabilities

# Load the saved model
hidden_channels = 1000
conv = GATv2Conv
heads = 1
dropout = 0.1
model = Model( conv, hidden_channels, heads, dropout)
model.load_state_dict(torch.load(f"{path_work}/ficheros_28032023/GATv2.1.model.single_batch.2707.pt"))


<All keys matched successfully>

> Make the edges match 

In [10]:
def get_nodes_id(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(ID_nodes_B1_r[tup[0]] , ID_nodes_A_r[tup[1]]) for tup in B1A_index_file]
    return id_file

> Generate embeddings DF

In [11]:
df_embeddings = all_data[["index"] + list(range(1,1281))]
df_embeddings = df_embeddings.drop_duplicates(subset = ["index"], keep = "first")
df_embeddings = df_embeddings.set_index(["index"])

In [12]:
# Getting the nodes A features : 
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
index_interest = []
for ancestor in instances_bacteria.Infected_ancestor :
    index = ID_nodes_A[ancestor]
    index_interest.append(index)
    
tensor_interest = [graph_data.x_dict["A"][i] for _,i in enumerate(index_interest)]
stacked_tensor = torch.stack(tensor_interest)
dico_kltype = {tuple(graph_data.x_dict["A"][i].numpy()) : all_data[all_data["Infected_ancestor"] == ID_nodes_A_r[i]]["KL_type_LCA"].values[0]  for _,i in enumerate(index_interest)}

def graph_single_Dpo_pred(df_embeddings) : 
    pred_data_single = HeteroData()
    # Defining the nodes :
    l_dpos = len(df_embeddings)
    pred_data_single["A"].x = stacked_tensor
    pred_data_single["B1"].x = torch.empty((l_dpos, 0))
    pred_data_single["B2"].x = torch.tensor(df_embeddings.iloc[:, :1280].values , dtype=torch.float)
    # Defining the edge_file :
    edge_index_B2_B1 = torch.tensor([[i , i] for i in range(l_dpos)] , dtype=torch.long)
    pred_data_single['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data_single["A"].x))] , dtype=torch.long)
    pred_data_single['B1', 'infects', 'A'].edge_label_index = edge_index_B1_A.t().contiguous()
    return pred_data_single

In [13]:
graph_ppt = graph_single_Dpo_pred(df_embeddings)

In [14]:
graph_ppt

HeteroData(
  A={ x=[127, 127] },
  B1={ x=[3493, 0] },
  B2={ x=[3493, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 3493] },
  (B1, infects, A)={ edge_label_index=[2, 443611] }
)

In [15]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return json.JSONEncoder.default(self, obj)
        
def get_nodes_id_single(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(df_embeddings.index[tup[0]] , dico_kltype[tuple(tensor_interest[tup[1]].numpy())]) for tup in B1A_index_file]
    return id_file
    
with open(f"{path_work}/PPT_Dpos.PPT_pred.1508.json", "w") as outfile :
    clean_results = {}
    predictions, probabilities = make_predictions(model, graph_ppt)
    ids = get_nodes_id_single(graph_ppt[("B1", "infects", "A")].edge_label_index)
    results = tuple(zip(ids,predictions.numpy(),probabilities.numpy()))
    positive_results = [pred for pred in results if int(pred[1]) == 1]
    for pos_res in positive_results : 
        prot = pos_res[0][0]
        kltype = pos_res[0][1]
        score = pos_res[2]
        a = {}
        a[kltype] = score
        if score > 0.0 : 
            #a = {"KLtype" : kltype , "Score" : score}
            if prot not in clean_results : 
                clean_results[prot] = a
            else :
                clean_results[prot].update(a)
    json.dump(clean_results , outfile, cls=CustomEncoder)

import pprint
pp = pprint.PrettyPrinter(width = 250, sort_dicts = True, compact = True)
#pp.pprint(clean_results)


In [37]:
pp.pprint(clean_results)

{'anubis__0': {'KL1': 0.7771984,
               'KL101': 0.85143304,
               'KL105': 0.7433367,
               'KL107': 0.56346005,
               'KL110': 0.7839924,
               'KL116': 0.9868101,
               'KL117': 0.8188118,
               'KL118': 0.9443864,
               'KL119': 0.52766716,
               'KL12': 0.82301766,
               'KL122': 0.6552289,
               'KL124': 0.9604773,
               'KL125': 0.98986125,
               'KL126': 0.79785097,
               'KL127': 0.5715543,
               'KL13': 0.5631354,
               'KL134': 0.55253345,
               'KL136': 0.5357025,
               'KL137': 0.8311151,
               'KL139': 0.5503703,
               'KL140': 0.96668935,
               'KL142': 0.95843506,
               'KL148': 0.54143506,
               'KL149': 0.6778512,
               'KL150': 0.61207914,
               'KL151': 0.97194517,
               'KL152': 0.54700476,
               'KL153': 0.7422077,
           

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [19]:
clean_results["anubis__1458"]

{'KL43': 0.96916693,
 'KL14': 0.97685224,
 'KL106': 0.8064071,
 'KL23': 0.87222654,
 'KL51': 0.98125666,
 'KL63': 0.95626915,
 'KL30': 0.993852,
 'KL64': 0.904456,
 'KL47': 0.79691035,
 'KL36': 0.8138072,
 'KL15': 0.9874321,
 'KL105': 0.79143906,
 'KL103': 0.93185574,
 'KL111': 0.8771114,
 'KL24': 0.9562845,
 'KL19': 0.9903784,
 'KL21': 0.9895074,
 'KL27': 0.9521645,
 'KL2': 0.98186535,
 'KL38': 0.9458575,
 'KL28': 0.948413,
 'KL52': 0.9907788,
 'KL108': 0.50933653,
 'KL155': 0.9040032,
 'KL112': 0.9599457,
 'KL46': 0.9710783,
 'KL102': 0.83005095,
 'KL1': 0.93821925,
 'KL17': 0.6660937,
 'KL151': 0.98106456,
 'KL22': 0.9766569,
 'KL8': 0.9936919,
 'KL16': 0.9980685,
 'KL3': 0.94445825,
 'KL110': 0.9179573,
 'KL25': 0.92315716,
 'KL163': 0.99621636,
 'KL10': 0.9730532,
 'KL48': 0.9779539,
 'KL57': 0.98630667,
 'KL45': 0.85927474,
 'KL58': 0.99833137,
 'KL132': 0.9644836,
 'KL113': 0.7474936,
 'KL127': 0.9791545,
 'KL122': 0.99719965,
 'KL142': 0.99461913,
 'KL62': 0.98816794,
 'KL66': 

In [18]:
from collections import Counter

to_check = {}
lengths = []
for dpo in clean_results : 
    to_check[dpo] = len(clean_results[dpo])
    lengths.append(len(clean_results[dpo]))

for dpo in to_check : 
    if to_check[dpo] > 100 : 
        print(dpo)

ppt__864
anubis__217
ppt__1169
ppt__1236
ppt__1579
anubis__315
ppt__1927
ppt__1980
ppt__1996
ppt__1153
anubis__493
ppt__3107
ppt__3415
ppt__3512
ppt__3644
ppt__3680
ppt__3702
ppt__1823
ppt__4016
ppt__4063
ppt__4064
ppt__4080
anubis__868
ppt__4330
ppt__4366
ppt__4427
ppt__4533
ppt__4795
ppt__5128
ppt__5375
ppt__5710
anubis__1315
minibatch__1996
ppt__7006
anubis__825
anubis__405
anubis__676
anubis__965
anubis__1112
anubis__1298
anubis__1311
anubis__1458


In [24]:
results_phage = []
for link in results : 
    KLtype = all_data[all_data["Infected_ancestor"] == link[0][1]]["KL_type_LCA"].values[0]
    line = (link[0][0].split("to_predict_")[1],KLtype, link[0][1], link[1],link[2])
    results_phage.append(line)
df = pd.DataFrame(results_phage, columns=['Dpo_index', 'KLtype','Ancestor','Prediction','Score'])

In [27]:
df = df[df["Score"] >= 0.95]
df

,Dpo_index,KLtype,Ancestor,Prediction,Score
0,ppt__2930,KL43,GCF_016651625.1,1.0,0.979078
1,ppt__2930,KL43,n1471,1.0,0.979078
2,ppt__2930,KL14,GCF_004313505.1,1.0,0.952483
3,ppt__2930,KL14,n830830,1.0,0.952483
134,ppt__2930,KL51,n3400,1.0,0.975307
...,...,...,...,...,...
4523,ppt__2930,KL22,GCF_900505785.1,1.0,0.998671
4524,ppt__2930,KL30,GCF_900506655.1,1.0,0.990287
4525,ppt__2930,KL136,GCF_007955275.1,1.0,0.953100
4528,ppt__2930,KL124,GCF_900172635.1,1.0,0.986359


In [11]:
def get_nodes_id(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(ID_nodes_B1_r[tup[0]] , ID_nodes_A_r[tup[1]]) for tup in B1A_index_file]
    return id_file

test = get_nodes_id(graph_data[("B1", "infects", "A")]["edge_label_index"])
